# Chapter 8: Molecules - Building and Manipulating

## Learning Objectives
- Build molecules from scratch using coordinates
- Use internal coordinates (z-matrix)
- Manipulate conformations (dihedral rotations)
- Align and superimpose molecules

---

## 8.1 Representing Molecules

Molecules can be represented using:
1. **Cartesian coordinates**: (x, y, z) for each atom
2. **Internal coordinates (Z-matrix)**: bonds, angles, dihedrals
3. **Connectivity**: which atoms are bonded

Each representation has advantages for different tasks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple, Dict, Optional, Set
from dataclasses import dataclass, field

# Covalent radii for bond detection (in Å)
COVALENT_RADII = {
    'H': 0.31, 'C': 0.76, 'N': 0.71, 'O': 0.66, 'F': 0.57,
    'P': 1.07, 'S': 1.05, 'Cl': 1.02, 'Br': 1.20, 'I': 1.39,
}

ELEMENT_COLORS = {
    'H': 'white', 'C': 'gray', 'N': 'blue', 'O': 'red',
    'F': 'yellowgreen', 'S': 'yellow', 'P': 'orange',
    'Cl': 'lime', 'Br': 'brown', 'I': 'purple',
}

@dataclass
class Atom:
    """Atom with element and position."""
    symbol: str
    position: np.ndarray
    index: int = 0
    
    @property
    def x(self) -> float:
        return self.position[0]
    
    @property
    def y(self) -> float:
        return self.position[1]
    
    @property
    def z(self) -> float:
        return self.position[2]

class Molecule:
    """A molecule with atoms and connectivity."""
    
    def __init__(self, name: str = "molecule"):
        self.name = name
        self.atoms: List[Atom] = []
        self.bonds: Set[Tuple[int, int]] = set()  # Set of (i, j) pairs
    
    def add_atom(self, symbol: str, position: np.ndarray) -> int:
        """Add atom and return its index."""
        idx = len(self.atoms)
        self.atoms.append(Atom(symbol, np.array(position, dtype=float), idx))
        return idx
    
    def add_bond(self, i: int, j: int) -> None:
        """Add bond between atoms i and j."""
        if i > j:
            i, j = j, i
        self.bonds.add((i, j))
    
    @property
    def n_atoms(self) -> int:
        return len(self.atoms)
    
    @property
    def positions(self) -> np.ndarray:
        """Get all positions as Nx3 array."""
        return np.array([a.position for a in self.atoms])
    
    @positions.setter
    def positions(self, new_positions: np.ndarray) -> None:
        """Set all positions."""
        for atom, pos in zip(self.atoms, new_positions):
            atom.position = np.array(pos, dtype=float)
    
    @property
    def symbols(self) -> List[str]:
        return [a.symbol for a in self.atoms]
    
    @property
    def center_of_mass(self) -> np.ndarray:
        """Calculate center of mass."""
        masses = {'H': 1.008, 'C': 12.011, 'N': 14.007, 'O': 15.999,
                  'F': 18.998, 'S': 32.065, 'P': 30.974, 'Cl': 35.453}
        total_mass = 0
        com = np.zeros(3)
        for atom in self.atoms:
            m = masses.get(atom.symbol, 12.0)
            com += m * atom.position
            total_mass += m
        return com / total_mass
    
    def auto_bonds(self, tolerance: float = 1.3) -> None:
        """Automatically detect bonds based on distance."""
        self.bonds.clear()
        for i, atom_i in enumerate(self.atoms):
            r_i = COVALENT_RADII.get(atom_i.symbol, 1.5)
            for j, atom_j in enumerate(self.atoms[i+1:], i+1):
                r_j = COVALENT_RADII.get(atom_j.symbol, 1.5)
                max_dist = (r_i + r_j) * tolerance
                dist = np.linalg.norm(atom_i.position - atom_j.position)
                if dist < max_dist:
                    self.add_bond(i, j)
    
    def translate(self, vector: np.ndarray) -> 'Molecule':
        """Translate molecule by vector."""
        for atom in self.atoms:
            atom.position += vector
        return self
    
    def center(self) -> 'Molecule':
        """Move center of mass to origin."""
        return self.translate(-self.center_of_mass)
    
    def copy(self) -> 'Molecule':
        """Create a deep copy."""
        mol = Molecule(self.name)
        for atom in self.atoms:
            mol.add_atom(atom.symbol, atom.position.copy())
        mol.bonds = self.bonds.copy()
        return mol
    
    def __repr__(self):
        formula = {}
        for atom in self.atoms:
            formula[atom.symbol] = formula.get(atom.symbol, 0) + 1
        formula_str = ''.join(f"{k}{v if v > 1 else ''}" for k, v in sorted(formula.items()))
        return f"Molecule('{self.name}', {formula_str})"

print("Molecule class defined!")

## 8.2 Building Molecules from Coordinates

In [ ]:
# Build water molecule
def build_water(bond_length: float = 0.96, angle: float = 104.5) -> Molecule:
    """Build H2O molecule.
    
    Args:
        bond_length: O-H bond length in Å
        angle: H-O-H angle in degrees
    """
    mol = Molecule("water")
    half_angle = np.radians(angle / 2)
    
    # Oxygen at origin
    mol.add_atom('O', [0, 0, 0])
    
    # Hydrogens
    mol.add_atom('H', [bond_length * np.sin(half_angle), 
                       bond_length * np.cos(half_angle), 0])
    mol.add_atom('H', [-bond_length * np.sin(half_angle),
                       bond_length * np.cos(half_angle), 0])
    
    mol.add_bond(0, 1)
    mol.add_bond(0, 2)
    
    return mol

# Build methane
def build_methane(bond_length: float = 1.09) -> Molecule:
    """Build CH4 with tetrahedral geometry."""
    mol = Molecule("methane")
    
    # Carbon at origin
    mol.add_atom('C', [0, 0, 0])
    
    # Tetrahedral positions
    tet_angle = np.arccos(-1/3)  # ~109.47°
    
    positions = [
        [0, 0, 1],  # Top
        [np.sqrt(8/9), 0, -1/3],
        [-np.sqrt(2/9), np.sqrt(2/3), -1/3],
        [-np.sqrt(2/9), -np.sqrt(2/3), -1/3]
    ]
    
    for pos in positions:
        h_pos = np.array(pos) * bond_length
        idx = mol.add_atom('H', h_pos)
        mol.add_bond(0, idx)
    
    return mol

# Build benzene
def build_benzene(cc_bond: float = 1.40, ch_bond: float = 1.09) -> Molecule:
    """Build C6H6 benzene ring."""
    mol = Molecule("benzene")
    
    # Hexagonal ring of carbons
    for i in range(6):
        angle = i * np.pi / 3
        x = cc_bond * np.cos(angle)
        y = cc_bond * np.sin(angle)
        mol.add_atom('C', [x, y, 0])
    
    # Add C-C bonds
    for i in range(6):
        mol.add_bond(i, (i + 1) % 6)
    
    # Add hydrogens
    for i in range(6):
        angle = i * np.pi / 3
        x = (cc_bond + ch_bond) * np.cos(angle)
        y = (cc_bond + ch_bond) * np.sin(angle)
        h_idx = mol.add_atom('H', [x, y, 0])
        mol.add_bond(i, h_idx)
    
    return mol

water = build_water()
methane = build_methane()
benzene = build_benzene()

print(water)
print(methane)
print(benzene)

## 8.3 Internal Coordinates (Z-Matrix)

A **Z-matrix** describes a molecule using:
- Bond lengths (distance from reference atom)
- Bond angles (angle with two reference atoms)
- Dihedral angles (torsion with three reference atoms)

This is natural for building molecules and changing conformations.

In [ ]:
def place_atom_from_zmatrix(positions: np.ndarray, bond_length: float,
                             angle: float = None, dihedral: float = None,
                             ref1: int = None, ref2: int = None, ref3: int = None) -> np.ndarray:
    """Place an atom using Z-matrix style coordinates.
    
    Args:
        positions: Existing atom positions (Nx3)
        bond_length: Distance from ref1
        angle: Angle ref2-ref1-new (degrees)
        dihedral: Dihedral ref3-ref2-ref1-new (degrees)
        ref1, ref2, ref3: Reference atom indices
    """
    if ref1 is None:
        # First atom at origin
        return np.array([0, 0, 0])
    
    if ref2 is None:
        # Second atom along z-axis
        return positions[ref1] + np.array([0, 0, bond_length])
    
    if ref3 is None:
        # Third atom in xz-plane
        angle_rad = np.radians(angle)
        v = positions[ref2] - positions[ref1]
        v = v / np.linalg.norm(v)
        
        # Find perpendicular vector
        if abs(v[0]) < 0.9:
            perp = np.array([1, 0, 0])
        else:
            perp = np.array([0, 1, 0])
        perp = perp - np.dot(perp, v) * v
        perp = perp / np.linalg.norm(perp)
        
        new_pos = (positions[ref1] + 
                   bond_length * np.cos(np.pi - angle_rad) * v +
                   bond_length * np.sin(np.pi - angle_rad) * perp)
        return new_pos
    
    # General case with dihedral
    angle_rad = np.radians(angle)
    dihedral_rad = np.radians(dihedral)
    
    # Bond vectors
    v1 = positions[ref2] - positions[ref1]  # ref1 -> ref2
    v2 = positions[ref1] - positions[ref2]  # ref2 -> ref1
    v3 = positions[ref3] - positions[ref2]  # ref2 -> ref3
    
    # Normalize ref1-ref2 vector
    n = v2 / np.linalg.norm(v2)
    
    # Build local coordinate system
    # n: along ref2-ref1
    # d: perpendicular, in plane of ref3-ref2-ref1
    d = v3 - np.dot(v3, n) * n
    d = d / np.linalg.norm(d)
    
    # e: perpendicular to both
    e = np.cross(n, d)
    
    # New position
    new_pos = (positions[ref1] +
               bond_length * np.cos(np.pi - angle_rad) * n +
               bond_length * np.sin(np.pi - angle_rad) * np.cos(dihedral_rad) * d +
               bond_length * np.sin(np.pi - angle_rad) * np.sin(dihedral_rad) * e)
    
    return new_pos

def build_from_zmatrix(zmat_data: List[Tuple]) -> Molecule:
    """Build molecule from Z-matrix specification.
    
    Each entry: (symbol, ref1, bond, ref2, angle, ref3, dihedral)
    For first atoms, omit undefined values.
    """
    mol = Molecule()
    positions = []
    
    for entry in zmat_data:
        symbol = entry[0]
        
        if len(entry) == 1:
            pos = place_atom_from_zmatrix(np.array(positions), 0)
        elif len(entry) == 3:
            pos = place_atom_from_zmatrix(np.array(positions), entry[2], ref1=entry[1])
        elif len(entry) == 5:
            pos = place_atom_from_zmatrix(np.array(positions), entry[2], entry[4],
                                          ref1=entry[1], ref2=entry[3])
        else:
            pos = place_atom_from_zmatrix(np.array(positions), entry[2], entry[4], entry[6],
                                          ref1=entry[1], ref2=entry[3], ref3=entry[5])
        
        positions.append(pos)
        mol.add_atom(symbol, pos)
    
    mol.auto_bonds()
    return mol

# Build ethane using Z-matrix
ethane_zmat = [
    ('C',),                              # C1 at origin
    ('C', 0, 1.54),                      # C2, bonded to C1
    ('H', 0, 1.09, 1, 109.5),            # H on C1
    ('H', 0, 1.09, 1, 109.5, 2, 120),    # H on C1
    ('H', 0, 1.09, 1, 109.5, 2, -120),   # H on C1
    ('H', 1, 1.09, 0, 109.5, 2, 60),     # H on C2 (staggered)
    ('H', 1, 1.09, 0, 109.5, 2, 180),    # H on C2
    ('H', 1, 1.09, 0, 109.5, 2, -60),    # H on C2
]

ethane = build_from_zmatrix(ethane_zmat)
print(ethane)

## 8.4 Molecular Geometry Analysis

In [ ]:
def distance(mol: Molecule, i: int, j: int) -> float:
    """Calculate distance between atoms i and j."""
    return np.linalg.norm(mol.atoms[i].position - mol.atoms[j].position)

def angle(mol: Molecule, i: int, j: int, k: int) -> float:
    """Calculate angle i-j-k in degrees."""
    v1 = mol.atoms[i].position - mol.atoms[j].position
    v2 = mol.atoms[k].position - mol.atoms[j].position
    
    cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.degrees(np.arccos(np.clip(cos_angle, -1, 1)))

def dihedral(mol: Molecule, i: int, j: int, k: int, l: int) -> float:
    """Calculate dihedral angle i-j-k-l in degrees."""
    b1 = mol.atoms[j].position - mol.atoms[i].position
    b2 = mol.atoms[k].position - mol.atoms[j].position
    b3 = mol.atoms[l].position - mol.atoms[k].position
    
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    
    n1 = n1 / np.linalg.norm(n1)
    n2 = n2 / np.linalg.norm(n2)
    
    m1 = np.cross(n1, b2 / np.linalg.norm(b2))
    
    x = np.dot(n1, n2)
    y = np.dot(m1, n2)
    
    return np.degrees(np.arctan2(y, x))

# Analyze ethane geometry
print("Ethane geometry:")
print(f"  C-C bond: {distance(ethane, 0, 1):.3f} Å")
print(f"  C-H bond: {distance(ethane, 0, 2):.3f} Å")
print(f"  H-C-H angle: {angle(ethane, 2, 0, 3):.1f}°")
print(f"  H-C-C angle: {angle(ethane, 2, 0, 1):.1f}°")
print(f"  H-C-C-H dihedral: {dihedral(ethane, 2, 0, 1, 5):.1f}°")

## 8.5 Conformational Changes

Rotation around bonds changes molecular conformation.

In [ ]:
def rotation_matrix_axis(axis: np.ndarray, theta: float) -> np.ndarray:
    """Create rotation matrix for rotation around an axis.
    
    Uses Rodrigues' formula.
    """
    axis = axis / np.linalg.norm(axis)
    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ])
    return np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * K @ K

def rotate_dihedral(mol: Molecule, bond: Tuple[int, int], 
                     delta_angle: float, fixed_end: int = 0) -> Molecule:
    """Rotate part of molecule around a bond.
    
    Args:
        mol: Molecule to modify
        bond: (i, j) atom indices of the bond to rotate around
        delta_angle: Rotation angle in degrees
        fixed_end: Which end to keep fixed (0 or 1)
    
    Returns:
        Modified molecule
    """
    new_mol = mol.copy()
    i, j = bond
    
    if fixed_end == 1:
        i, j = j, i  # Swap so we rotate around j
    
    # Find atoms to rotate (connected to j but not through i)
    to_rotate = set()
    visited = {i}  # Don't cross the bond
    stack = [j]
    
    while stack:
        current = stack.pop()
        if current in visited:
            continue
        visited.add(current)
        to_rotate.add(current)
        
        # Find connected atoms
        for bond in mol.bonds:
            if current in bond:
                neighbor = bond[1] if bond[0] == current else bond[0]
                if neighbor not in visited:
                    stack.append(neighbor)
    
    # Rotation axis and origin
    axis = mol.atoms[j].position - mol.atoms[i].position
    origin = mol.atoms[i].position
    
    # Rotation matrix
    R = rotation_matrix_axis(axis, np.radians(delta_angle))
    
    # Rotate selected atoms
    for idx in to_rotate:
        pos = new_mol.atoms[idx].position - origin
        new_mol.atoms[idx].position = R @ pos + origin
    
    return new_mol

# Generate ethane conformations
def generate_conformations(mol: Molecule, bond: Tuple[int, int],
                            n_conformers: int = 12) -> List[Molecule]:
    """Generate conformations by rotating around bond."""
    conformers = []
    for i in range(n_conformers):
        angle = i * 360 / n_conformers
        conf = rotate_dihedral(mol, bond, angle)
        conf.name = f"conformer_{angle:.0f}"
        conformers.append(conf)
    return conformers

# Generate ethane conformers
ethane_conformers = generate_conformations(ethane, (0, 1), n_conformers=12)
print(f"Generated {len(ethane_conformers)} conformers")

## 8.6 Visualizing Molecules

In [ ]:
def plot_molecule(mol: Molecule, ax: plt.Axes = None, 
                   show_bonds: bool = True) -> plt.Axes:
    """Plot a molecule in 3D."""
    if ax is None:
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')
    
    # Plot atoms
    for atom in mol.atoms:
        color = ELEMENT_COLORS.get(atom.symbol, 'gray')
        size = COVALENT_RADII.get(atom.symbol, 1.0) * 500
        ax.scatter(*atom.position, s=size, c=color, 
                   edgecolors='black', alpha=0.9)
    
    # Plot bonds
    if show_bonds:
        for i, j in mol.bonds:
            pos_i = mol.atoms[i].position
            pos_j = mol.atoms[j].position
            ax.plot3D(*zip(pos_i, pos_j), 'k-', linewidth=2)
    
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.set_title(mol.name)
    
    # Equal aspect ratio
    positions = mol.positions
    max_range = np.max(np.ptp(positions, axis=0)) / 2 + 0.5
    center = np.mean(positions, axis=0)
    ax.set_xlim([center[0] - max_range, center[0] + max_range])
    ax.set_ylim([center[1] - max_range, center[1] + max_range])
    ax.set_zlim([center[2] - max_range, center[2] + max_range])
    
    return ax

# Plot molecules
fig = plt.figure(figsize=(15, 5))

for i, (mol, title) in enumerate([
    (water, "Water"),
    (methane, "Methane"),
    (benzene, "Benzene")
]):
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    mol.name = title
    plot_molecule(mol, ax)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize ethane conformations (staggered vs eclipsed)
staggered = ethane_conformers[0]
staggered.name = "Staggered (0°)"

eclipsed = ethane_conformers[6]  # 180° from staggered
eclipsed.name = "Eclipsed (180°)"

fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='3d')
plot_molecule(staggered, ax1)
ax1.view_init(elev=0, azim=0)  # Look along C-C bond

ax2 = fig.add_subplot(122, projection='3d')
plot_molecule(eclipsed, ax2)
ax2.view_init(elev=0, azim=0)

plt.tight_layout()
plt.show()

## 8.7 Molecular Alignment (RMSD)

To compare conformations, we align molecules and compute the **RMSD** (Root Mean Square Deviation).

In [ ]:
def rmsd(mol1: Molecule, mol2: Molecule) -> float:
    """Calculate RMSD between two molecules."""
    if mol1.n_atoms != mol2.n_atoms:
        raise ValueError("Molecules must have same number of atoms")
    
    diff = mol1.positions - mol2.positions
    return np.sqrt(np.mean(np.sum(diff**2, axis=1)))

def kabsch_rotation(P: np.ndarray, Q: np.ndarray) -> np.ndarray:
    """Find optimal rotation to align P onto Q using Kabsch algorithm.
    
    Args:
        P, Q: Nx3 arrays of coordinates (centered at origin)
    
    Returns:
        3x3 rotation matrix
    """
    # Cross-covariance matrix
    H = P.T @ Q
    
    # SVD
    U, S, Vt = np.linalg.svd(H)
    
    # Rotation matrix
    d = np.linalg.det(Vt.T @ U.T)
    R = Vt.T @ np.diag([1, 1, d]) @ U.T
    
    return R

def align_molecules(mobile: Molecule, target: Molecule) -> Molecule:
    """Align mobile molecule onto target using Kabsch algorithm.
    
    Returns:
        New aligned molecule
    """
    aligned = mobile.copy()
    
    # Center both
    P = mobile.positions - mobile.center_of_mass
    Q = target.positions - target.center_of_mass
    
    # Find optimal rotation
    R = kabsch_rotation(P, Q)
    
    # Apply rotation and translation
    aligned_positions = (P @ R.T) + target.center_of_mass
    aligned.positions = aligned_positions
    
    return aligned

# Compare staggered and eclipsed ethane
aligned_eclipsed = align_molecules(eclipsed, staggered)
print(f"RMSD (staggered vs eclipsed): {rmsd(staggered, aligned_eclipsed):.3f} Å")

## 8.8 Building Complex Molecules

Let's build a more complex molecule: butane (C₄H₁₀).

In [ ]:
def build_alkane(n_carbons: int, bond_length: float = 1.54,
                  ch_bond: float = 1.09, angle: float = 109.5,
                  dihedral: float = 180) -> Molecule:
    """Build a linear alkane chain.
    
    Args:
        n_carbons: Number of carbon atoms
        dihedral: C-C-C-C dihedral angle (180 = all-trans)
    """
    mol = Molecule(f"C{n_carbons}H{2*n_carbons+2}")
    
    if n_carbons < 1:
        return mol
    
    # Build carbon backbone
    zmat = [('C',)]  # First carbon
    
    if n_carbons >= 2:
        zmat.append(('C', 0, bond_length))
    
    for i in range(2, n_carbons):
        zmat.append(('C', i-1, bond_length, i-2, angle, i-3 if i > 2 else 0, dihedral))
    
    # Build backbone
    positions = []
    for entry in zmat:
        if len(entry) == 1:
            pos = place_atom_from_zmatrix(np.array(positions), 0)
        elif len(entry) == 3:
            pos = place_atom_from_zmatrix(np.array(positions), entry[2], ref1=entry[1])
        elif len(entry) == 5:
            pos = place_atom_from_zmatrix(np.array(positions), entry[2], entry[4],
                                          ref1=entry[1], ref2=entry[3])
        else:
            pos = place_atom_from_zmatrix(np.array(positions), entry[2], entry[4], entry[6],
                                          ref1=entry[1], ref2=entry[3], ref3=entry[5])
        positions.append(pos)
        mol.add_atom('C', pos)
    
    positions = np.array(positions)
    
    # Add C-C bonds
    for i in range(n_carbons - 1):
        mol.add_bond(i, i + 1)
    
    # Add hydrogens to each carbon with proper tetrahedral geometry
    for c_idx in range(n_carbons):
        # Determine number of H atoms needed
        if c_idx == 0 or c_idx == n_carbons - 1:
            n_h = 3  # Terminal carbon
        else:
            n_h = 2  # Middle carbon
        
        # Get reference directions
        c_pos = positions[c_idx]
        
        if c_idx == 0:
            bond_dir = positions[1] - c_pos
        elif c_idx == n_carbons - 1:
            bond_dir = positions[c_idx - 1] - c_pos
        else:
            # For middle carbons, use average of bond directions
            bond_dir = positions[c_idx - 1] - c_pos
        
        bond_dir = bond_dir / np.linalg.norm(bond_dir)
        
        # Create perpendicular vectors
        if abs(bond_dir[0]) < 0.9:
            perp1 = np.cross(bond_dir, [1, 0, 0])
        else:
            perp1 = np.cross(bond_dir, [0, 1, 0])
        perp1 = perp1 / np.linalg.norm(perp1)
        perp2 = np.cross(bond_dir, perp1)
        
        # Place hydrogens
        tet_angle = np.radians(109.5)
        
        for h_i in range(n_h):
            phi = 2 * np.pi * h_i / n_h
            h_dir = (
                -np.cos(tet_angle) * bond_dir +
                np.sin(tet_angle) * (np.cos(phi) * perp1 + np.sin(phi) * perp2)
            )
            h_pos = c_pos + ch_bond * h_dir
            h_idx = mol.add_atom('H', h_pos)
            mol.add_bond(c_idx, h_idx)
    
    return mol

# Build butane
butane = build_alkane(4, dihedral=180)  # All-trans
print(butane)

# Plot
plot_molecule(butane)
plt.show()

---

## Practice Exercises

### Exercise 8.1: Build Ammonia
Build an ammonia (NH₃) molecule with correct pyramidal geometry.

In [ ]:
# YOUR CODE HERE
def build_ammonia(nh_bond: float = 1.01, angle: float = 107.0) -> Molecule:
    """Build NH3 with pyramidal geometry."""
    # TODO: Implement
    pass

### Exercise 8.2: Torsional Energy Scan

Write a function to generate structures for a torsional energy scan.

In [ ]:
# YOUR CODE HERE
def torsional_scan(mol: Molecule, bond: Tuple[int, int],
                   ref_atoms: Tuple[int, int, int, int],
                   angles: List[float]) -> List[Tuple[float, Molecule]]:
    """Generate structures for torsional scan.
    
    Args:
        mol: Starting molecule
        bond: Bond to rotate around
        ref_atoms: (i, j, k, l) atoms defining the dihedral
        angles: List of target dihedral angles
    
    Returns:
        List of (angle, molecule) pairs
    """
    # TODO: Implement
    pass

### Exercise 8.3: Ring Builder

Build a cyclohexane ring in chair conformation.

In [ ]:
# YOUR CODE HERE
def build_cyclohexane_chair(cc_bond: float = 1.54, ch_bond: float = 1.09) -> Molecule:
    """Build cyclohexane in chair conformation.
    
    Chair conformation: alternating up/down carbons
    """
    # TODO: Implement
    pass

---

## Key Takeaways

1. **Cartesian and internal coordinates** serve different purposes
2. **Z-matrices** are natural for building and conformational changes
3. **Dihedral rotation** preserves bond lengths and angles
4. **RMSD and alignment** are essential for structure comparison
5. **Auto-bonding** based on covalent radii works for most cases

## Next Chapter Preview

In Chapter 9, we'll explore **advanced transformations** including supercell construction and strain application.